In [1]:
!pip install google-adk

  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached sqlalchemy-2.0.40-cp313-cp313-win_amd64.whl.metadata (9.9 kB)
  Using cached cryptography-44.0.2-cp39-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached shapely-2.1.0-cp313-cp313-win_amd64.whl.metadata (7.0 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.8.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached greenlet-3.1.1-cp313-cp313-win_amd64.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached s

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aperturedb 0.4.45 requires grpcio-status==1.48.2, but you have grpcio-status 1.71.0 which is incompatible.


In [2]:
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import LlmAgent
from pydantic import BaseModel, Field

class EntityOutput(BaseModel):
    """Output model for entity extraction agent."""
    entity_type: str = Field(..., description="Type of the entity (e.g., Person, Organization).")
    properties: list[str] = Field(..., description="List of properties associated with the entity type.")

entity_instructions = """Workflow: 
You are part of an agentic workflow that processes data input by a user step by step. The end result of the workflow is a detailed knowledge graph generated from the input data.
Following are the agents in this workflow:

1. Preprocessing & Entity Extraction Agent:
Extracts general entity categories (e.g., Person, Company, Location) and their possible properties from the raw text.
Responsibilities:
- Identify entity types and associated properties (e.g., name, age, industry).
- Return a global entity list for further processing.

2. Entity Instance Extraction Agents:
Process text chunks in parallel to extract instances of entities (e.g., John Doe, Acme Corp) and fill in their properties.
Responsibilities:
- Extract entity instances from text chunks.
- Return instances with defined properties.

3. Duplicate Removal & ID Assignment Agent:
Removes duplicate instances and assigns unique IDs to each entity instance.
Responsibilities:
- Remove duplicates and retain the most complete instance.
- Assign unique IDs to entities.

4. Global Relationship Extraction Agent:
Extracts relationships between entities using the full text and the finalized entity list.
Responsibilities:
- Identify relationships (e.g., works_at) between entities.
- Return relationships with entities referenced by unique IDs.

Your Role: Preprocessing & Entity Extraction Agent

Your Task:
Given the following text, extract broad, general categories of entities (e.g., people, organizations, locations, concepts, events, technologies) and list the general types of properties associated with each category. 

Please focus on identifying general categories of entities and generalizable properties that could apply to those categories.

Example Entity Types & Properties (These are just examples, please generalize): 
- Person: name, age, occupation, address, etc.
- Organization: name, type, location, industry, etc.
- Location: country, city, coordinates, description, etc.
- Technology: name, type, purpose, capabilities, etc.
- Event: name, date, location, significance, etc.

Response format:

json
{
  "EntityType1": ["property1", "property2", "property3", ...],
  "EntityType2": ["property1", "property2", "property3", ...],
  "EntityType3": ["property1", "property2", "property3", ...],
  ...
}"""

# Example: Adding instructions
entity_agent = LlmAgent(
    model="gemini-2.0-flash",
    name="entity_agent",
    description="Extracts general entity categories and their possible properties from the raw text.",
    instruction=entity_instructions,
    output_schema=EntityOutput, 
    output_key="entities",
    disallow_transfer_to_parent=True, 
    disallow_transfer_to_peers=True
)